https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python/

In [1]:
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2021-07-19 22:59:50.001646: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-19 22:59:50.001683: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/venv/lib/python3.8/site-packages/rl/agents/dqn.py line 108
-        if hasattr(model.output, '__len__') and len(model.output) > 1:
+        if hasattr(model.output, '__len__'):

In [2]:
ENV_NAME = 'CartPole-v0'

# Get the environment and extract the number of actions available in the Cartpole problem
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [72]:
env.action_space

Discrete(2)

In [70]:
env.observation_space

Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)

In [74]:
env.observation_space.shape

(4,)

In [57]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(16, activation="relu"))
# model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 34        
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 658
Trainable params: 658
Non-trainable params: 0
________________________________________________________

In [71]:
model.output.shape

TensorShape([None, 2])

In [58]:
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=50000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

/home/tttienthinh/Documents/Programmation/Scientist/GeneticAlgorithm/venv/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [63]:
# Okay, now it's time to learn something! We visualize the training here for show, but this slows down training quite a lot. 
dqn.fit(env, nb_steps=10_000, visualize=False, verbose=2)

Training for 10000 steps ...
  126/10000: episode: 1, duration: 0.767s, episode steps: 126, steps per second: 164, episode reward: 126.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.556 [0.000, 1.000],  loss: 3.994613, mae: 28.018042, mean_q: 56.470532
  253/10000: episode: 2, duration: 0.879s, episode steps: 127, steps per second: 144, episode reward: 127.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.551 [0.000, 1.000],  loss: 3.072160, mae: 28.036873, mean_q: 56.639156
  392/10000: episode: 3, duration: 0.940s, episode steps: 139, steps per second: 148, episode reward: 139.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.554 [0.000, 1.000],  loss: 2.938457, mae: 28.050371, mean_q: 56.579521
  592/10000: episode: 4, duration: 1.305s, episode steps: 200, steps per second: 153, episode reward: 200.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.525 [0.000, 1.000],  loss: 2.406415, mae: 28.269766, mean_q: 57.072075
  753/10000: episode: 5, durati

 6293/10000: episode: 36, duration: 1.249s, episode steps: 191, steps per second: 153, episode reward: 191.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.476 [0.000, 1.000],  loss: 4.957035, mae: 33.986820, mean_q: 68.533287
 6486/10000: episode: 37, duration: 1.307s, episode steps: 193, steps per second: 148, episode reward: 193.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.534 [0.000, 1.000],  loss: 3.753073, mae: 34.289268, mean_q: 69.209969
 6677/10000: episode: 38, duration: 1.252s, episode steps: 191, steps per second: 153, episode reward: 191.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.476 [0.000, 1.000],  loss: 4.172369, mae: 34.320705, mean_q: 69.228783
 6850/10000: episode: 39, duration: 1.115s, episode steps: 173, steps per second: 155, episode reward: 173.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.538 [0.000, 1.000],  loss: 4.242710, mae: 34.592770, mean_q: 69.747993
 7030/10000: episode: 40, duration: 1.176s, episode step

In [64]:
results = dqn.test(env, nb_episodes=100, visualize=False)
np.mean(results.history["episode_reward"])

Testing for 100 episodes ...
Episode 1: reward: 188.000, steps: 188
Episode 2: reward: 181.000, steps: 181
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 172.000, steps: 172
Episode 7: reward: 196.000, steps: 196
Episode 8: reward: 168.000, steps: 168
Episode 9: reward: 182.000, steps: 182
Episode 10: reward: 178.000, steps: 178
Episode 11: reward: 195.000, steps: 195
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 193.000, steps: 193
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 168.000, steps: 168
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 178.000, steps: 178
Episode 18: reward: 199.000, steps: 199
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 197.000, steps: 197
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 190.000, steps: 190
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

190.95

In [67]:
obs = env.reset()
action = dqn.model.predict([[[obs]]]).argmax()
tt_reward = 0
done = False
while not done:
    next_state, reward, done, _ = env.step(action)
    result = dqn.model.predict([[[next_state]]])
    action = result.argmax()
    print(result, action, done)
    env.render()
    tt_reward += reward
print(tt_reward)

[[98.962425 96.43396 ]] 0 False
[[99.99229 99.69212]] 0 False
[[99.091866 99.88295 ]] 1 False
[[100.281   100.25469]] 0 False
[[ 99.337814 100.38535 ]] 1 False
[[100.52338  100.741745]] 1 False
[[101.264656  98.528435]] 0 False
[[102.08593 101.90326]] 0 False
[[101.2743  102.14484]] 1 False
[[102.56575 102.58135]] 1 False
[[103.35154 100.07681]] 0 False
[[103.92098 103.48895]] 0 False
[[102.63896  103.204704]] 1 False
[[104.02851  103.723755]] 0 False
[[102.29484  102.952415]] 1 False
[[103.57693 103.34411]] 0 False
[[102.0173  102.72908]] 1 False
[[103.32009  103.081245]] 0 False
[[101.80416  102.533516]] 1 False
[[102.86048  102.449356]] 0 False
[[101.65482 102.36526]] 1 False
[[102.05963 101.23636]] 0 False
[[101.57032 102.22455]] 1 False
[[101.187325  99.850525]] 0 False
[[101.5533 102.1123]] 1 False
[[100.36224  98.25416]] 0 False
[[100.855225 100.868576]] 1 False
[[99.445305 96.420815]] 0 False
[[99.92192 99.17427]] 0 False
[[ 98.89958 100.25314]] 1 False
[[98.75707 98.38344]] 0 

In [68]:
dqn.model.save("2-Cartpole")

2021-07-19 23:32:14.004118: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-07-19 23:32:14.004189: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      


INFO:tensorflow:Assets written to: 2-Cartpole/assets
